In [39]:
import os
import numpy as np
import collections
import tensorflow as tf
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt 

In [40]:
# Path to the new balanced dataset
DATASET_PATH = Path(r"C:\Users\Raj Kumar\Desktop\Projects\ML Projects\Currency Detection\Datasets")

# Image parameters
IMAGE_SIZE = (128, 128)  # Adjust size based on model needs
BATCH_SIZE = 32
EPOCHS = 20


In [41]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% for validation
)

# Load training data
train_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# Load validation data
val_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Get number of classes
num_classes = len(train_data.class_indices)
print(f"Detected {num_classes} classes: {train_data.class_indices}")

Found 2148 images belonging to 11 classes.
Found 531 images belonging to 11 classes.
Detected 11 classes: {'fifty_new': 0, 'fifty_old': 1, 'five_hundred': 2, 'hundred_new': 3, 'hundred_old': 4, 'ten_new': 5, 'ten_old': 6, 'twenty_new': 7, 'twenty_old': 8, 'two_hundred': 9, 'two_thousand': 10}


In [42]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the MobileNetV2 model (without the top classification layer)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the base model layers (so they don't get modified in the first training phase)
base_model.trainable = False

# Build the new model
inputs = base_model.input
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Prevent overfitting
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(len(train_data.class_indices), activation='softmax')(x)  # Output layer

# Define the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 32, 32,    │      2,304 │ block_1_depthwis

 Total params: 3,048,011 (11.63 MB)

 Trainable params: 790,027 (3.01 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [43]:
history = model.fit(train_data, validation_data=val_data, epochs=20, verbose=1)


C:\Users\Raj Kumar\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - accuracy: 0.1142 - loss: 2.8426 - val_accuracy: 0.1940 - val_loss: 2.2754
Epoch 2/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 29s 422ms/step - accuracy: 0.2104 - loss: 2.2318 - val_accuracy: 0.2053 - val_loss: 2.1932
Epoch 3/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 30s 442ms/step - accuracy: 0.2731 - loss: 2.0376 - val_accuracy: 0.2919 - val_loss: 2.0782
Epoch 4/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 29s 430ms/step - accuracy: 0.3426 - loss: 1.8623 - val_accuracy: 0.2919 - val_loss: 2.0612
Epoch 5/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 36s 526ms/step - accuracy: 0.4082 - loss: 1.6972 - val_accuracy: 0.3183 - val_loss: 1.9846
Epoch 6/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 34s 494ms/step - accuracy: 0.4417 - loss: 1.5882 - val_accuracy: 0.3352 - val_loss: 1.9701
Epoch 7/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 34s 507ms/step - accuracy: 0.4843 - loss: 1.5370 - val_accuracy: 0.3503 - val_loss: 1.9658
Epoch 8/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 33s 479ms/step - accuracy: 0.5228 - loss: 1.3962 - val_accura

In [44]:
for layer in base_model.layers[-20:]:  
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

history_finetune = model.fit(train_data, validation_data=val_data, epochs=10, verbose=1)


Epoch 1/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 54s 636ms/step - accuracy: 0.4879 - loss: 1.4632 - val_accuracy: 0.3861 - val_loss: 1.9312
Epoch 2/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 36s 523ms/step - accuracy: 0.5367 - loss: 1.3099 - val_accuracy: 0.4218 - val_loss: 1.9276
Epoch 3/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 38s 554ms/step - accuracy: 0.5959 - loss: 1.1976 - val_accuracy: 0.3785 - val_loss: 2.0414
Epoch 4/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 35s 516ms/step - accuracy: 0.6076 - loss: 1.1491 - val_accuracy: 0.4030 - val_loss: 2.0439
Epoch 5/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 35s 510ms/step - accuracy: 0.6185 - loss: 1.0747 - val_accuracy: 0.4143 - val_loss: 1.9463
Epoch 6/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 36s 530ms/step - accuracy: 0.6667 - loss: 0.9765 - val_accuracy: 0.4030 - val_loss: 2.1045
Epoch 7/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 35s 513ms/step - accuracy: 0.6624 - loss: 1.0122 - val_accuracy: 0.4350 - val_loss: 1.9418
Epoch 8/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 35s 515ms/step - accuracy: 0.6616 - loss: 0.9532 - val_accu

In [45]:
model.save("pre_currency_model.h5")
print("Model saved successfully!")

Model saved successfully!


In [46]:
load_model = tf.keras.models.load_model
model = load_model("pre_currency_model.h5")
print("Model loaded successfully!")

Model loaded successfully!


In [47]:
loss, accuracy = model.evaluate(val_data)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

17/17 ━━━━━━━━━━━━━━━━━━━━ 14s 532ms/step - accuracy: 0.4209 - loss: 2.1500
Validation Accuracy: 42.56%


In [52]:
print(train_data.class_indices)

{'fifty_new': 0, 'fifty_old': 1, 'five_hundred': 2, 'hundred_new': 3, 'hundred_old': 4, 'ten_new': 5, 'ten_old': 6, 'twenty_new': 7, 'twenty_old': 8, 'two_hundred': 9, 'two_thousand': 10}


In [ ]:
currency_labels = [
    "₹50", "₹50", "₹500", "₹100", "₹100", 
    "₹10", "₹10", "20", "20", "200", "2000"
]

In [49]:
from tensorflow.keras.preprocessing import image
# Function to preprocess the image
def preprocess_image(img_path, target_size=(128, 128)):
    img = image.load_img(img_path, target_size=target_size)  # Load image
    img_array = image.img_to_array(img)  # Convert to array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dims for batch
    img_array = img_array / 255.0  # Normalize
    return img_array

In [50]:
def predict_currency(img_path):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)  # Get class with highest probability
    confidence = np.max(prediction)  # Get confidence score
    
    currency_name = currency_labels[predicted_class]  # Map index to currency name
    print(f"Predicted Currency: {currency_name} (Confidence: {confidence * 100:.2f}%)")
    return currency_name, confidence

In [51]:
img_path = r"C:\Users\Raj Kumar\Downloads\notes\2002.jpg"  # Change as needed
predict_currency(img_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Currency: 200 (Confidence: 99.38%)


('200', 0.9937628)

In [2]:
!pip install pyttsx3

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/241.5 kB ? eta -:--:--
   ------ --------------------------------- 41.0/241.5 kB ? eta -:--:--
   ------ --------------------------------- 41.0/241.5 kB ? eta -:--:--
   ---------------------------- ----------- 174.1/241.5 kB 1.7 MB/s eta 0:00:01
   ---------------------------- ----------- 174.1/241.5 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 241.5/241.5 kB 1.2 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import pyttsx3

def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

# Example usage
speak("This is a 500 rupee note")